In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [27]:
import asyncio
import math
import os
import time
from typing import List
import pandas as pd
import torch
from tqdm import tqdm
from src.utils import Prompt, acall_gpt, call_gpt, VICUNA_MODEL_PATH, ALPACA_MODEL_PATH, get_gpu_memory
from fastchat.model.model_adapter import get_conversation_template
from fastchat.serve.inference import load_model
from os_models import OSInit
import baseline_utils
from types import SimpleNamespace
import json


In [29]:
args = {
    'data_dir': '../../data/gsm_data',
    'save_dir': 'models',
    'debug': False,
    'exp_label': 'default',
    'task': '0cot_gsm',
    'model': 'alpaca',
    'max_tokens': 2048,
    'temperature': 0.0,
}
args['ckpt_path'] = os.path.join(args['save_dir'], args['exp_label'])
args = SimpleNamespace(**args)

In [25]:
def gsm_baseline(model, task):
    model.setup_prompt_from_examples_file(f'../../prompt/os_baseline/{task}.txt')
    
    for i in range(3):
        for variant in ['original', 'irc']:
            data = baseline_utils.load_gsm_data(os.path.join(args.data_dir, f'gsmic_mixed_{i}_{variant}.jsonl'))
            if (os.path.exists(os.path.join(args.save_dir, f'gsmic_mixed_{i}_{variant}_output_{args.model}_{task}.json')) 
                or os.path.exists(os.path.join(args.save_dir, f'hand_gsmic_mixed_{i}_{variant}_output_{args.model}_{task}.json'))):
                continue
            solutions = model([d['input'] for d in data])
            for i in range(len(data)):
                data[i]['output'] = solutions[i]
            output_file = os.path.join(args.save_dir, f'gsmic_mixed_{i}_{variant}_output_{args.model}_{task}.json')
            with open(output_file, 'w') as f:
                  f.write(json.dumps(data) + '\n')

In [11]:
data = baseline_utils.load_gsm_data(os.path.join(args.data_dir, f'gsmic_mixed_0_original.jsonl'))

In [12]:
data[:3]

[{'input': "Oli's banana split has 4 scoops of ice cream while Victoria has twice more scoops of ice cream than Oli's. How many more scoops of ice cream does Victoria have than Oli?",
  'target': '4',
  'n_steps': 2},
 {'input': "Mr. Angstadt has 120 students throughout the school day.  Half of them are enrolled in Statistics.  Of the students in Statistics, 90 percent are seniors.  How many of Mr. Angstadt's students are seniors enrolled in Statistics?",
  'target': '54',
  'n_steps': 2},
 {'input': 'Helga went shopping for a new pair of shoes. At the first store, she tried on 7 pairs of shoes. At the second store, she tried on 2 more pairs than at the first store. At the third store, she did not try on any shoes, but she did buy a scarf.  But at the fourth store, she tried on twice as many pairs of shoes as she did at all three other stores combined, before finally choosing a pair to buy.  What is the total number of pairs of shoes Helga tried on before buying her new shoes?',
  'tar

In [23]:
os_task_init = OSInit(
        prompt_examples=f'../../prompt/os_baseline/0cot_gsm.txt',
        engine=args.model,
        temperature=0.01
)

0,1,2


Loading checkpoint shards: 100%|██████████| 3/3 [00:24<00:00,  8.21s/it]


In [30]:

print(args.model, args.task)
gsm_baseline(os_task_init, args.task)

alpaca 0cot_gsm
['A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human\'s questions.\n### Human: What are the key differences between renewable and non-renewable energy sources?\n### Assistant: Renewable energy sources are those that can be replenished naturally in a relatively short amount of time, such as solar, wind, hydro, geothermal, and biomass. Non-renewable energy sources, on the other hand, are finite and will eventually be depleted, such as coal, oil, and natural gas. Here are some key differences between renewable and non-renewable energy sources:\n1. Availability: Renewable energy sources are virtually inexhaustible, while non-renewable energy sources are finite and will eventually run out.\n2. Environmental impact: Renewable energy sources have a much lower environmental impact than non-renewable sources, which can lead to air and water pollution, greenhouse gas emissions, and other 

  0%|          | 0/1 [00:00<?, ?it/s]

GPU Memory 0: 4.351450624 GB
GPU Memory 1: 4.895530496 GB
GPU Memory 2: 4.34184192 GB


100%|██████████| 1/1 [00:02<00:00,  2.15s/it]


['A chat between a curious human and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the human\'s questions.\n### Human: What are the key differences between renewable and non-renewable energy sources?\n### Assistant: Renewable energy sources are those that can be replenished naturally in a relatively short amount of time, such as solar, wind, hydro, geothermal, and biomass. Non-renewable energy sources, on the other hand, are finite and will eventually be depleted, such as coal, oil, and natural gas. Here are some key differences between renewable and non-renewable energy sources:\n1. Availability: Renewable energy sources are virtually inexhaustible, while non-renewable energy sources are finite and will eventually run out.\n2. Environmental impact: Renewable energy sources have a much lower environmental impact than non-renewable sources, which can lead to air and water pollution, greenhouse gas emissions, and other negative effects

  0%|          | 0/1 [00:00<?, ?it/s]

GPU Memory 0: 4.351450624 GB
GPU Memory 1: 4.895530496 GB
GPU Memory 2: 4.34184192 GB


100%|██████████| 1/1 [00:05<00:00,  5.23s/it]
